In [4]:
import numpy as np
import pandas as pd
import xarray as xr

from geopandas import GeoDataFrame
from shapely.wkb import loads
from tqdm import tqdm

In [5]:
df = pd.read_csv('../../data/finland/metsakeskus_sampleplots.csv')
df.head()

,sampleplotid,clusterid,sampleplottype,measurementdate,center_x,center_y,center_z,geom,mean_age_years,mean_dbh_cm,...,total_stems_ha,pine_stems_ha,spruce_stems_ha,birch_stems_ha,other_bl_stems_ha,total_m2_ha,pine_m2_ha,spruce_m2_ha,birch_m2_ha,other_bl_m2_ha
0,639971,48070005,1,2017-08-14,393888.00,7401168.00,0.00,0101000020FB0B0000FE020000800A1841E8FDFFFFB33B...,39,9.20,...,6090,550,196,5344,0,24.50,6.00,1.00,17.50,0.00
1,718519,47110091,1,2018-07-18,576046.89,7395714.46,346.79,0101000020FB0B00005B5A9FC75D9421413C7C729D6036...,66,16.41,...,1139,39,354,707,39,14.75,0.26,5.52,8.54,0.43
2,104636,40110069,1,2011-09-26,573195.59,7037602.47,197.03,0101000020FB0B0000D0C0052E177E21418A2A189EA8D8...,72,23.75,...,629,39,236,354,0,20.00,1.90,7.20,10.90,0.00
3,265558,43116119,1,2013-08-29,594209.26,7181267.18,255.95,0101000020FB0B00008B722D8542222241B77081CBF464...,42,17.99,...,1061,707,0,354,0,19.19,17.93,0.00,1.26,0.00
4,696784,39100744,1,2018-08-13,546924.17,6997684.98,124.56,0101000020FB0B0000579BDA54D8B020417A07993EADB1...,14,6.95,...,1690,118,1297,275,0,5.32,0.13,4.24,0.95,0.00


In [7]:
geometries = [loads(g, hex=True) for g in df['geom'].tolist()]

In [8]:
crs = {'init': 'epsg:3067'}

In [11]:
gdf = GeoDataFrame(df.drop('geom', axis=1), crs=crs, geometry=geometries)

In [12]:
gdf = gdf.to_crs({'init': 'epsg:4326'})

In [13]:
gdf['geometry'][0].xy

(array('d', [24.594444525970196]), array('d', [66.71062119422791]))

Load UERRA CDO indexes rasters

In [14]:
ds_cfd = xr.open_dataset("/data-uerra/mescan-surfex/temperature/mescan-surfex-nordics-eca_cfd.nc")
ds_fd = xr.open_dataset("/data-uerra/mescan-surfex/temperature/mescan-surfex-nordics-eca_fd.nc")
ds_csu = xr.open_dataset("/data-uerra/mescan-surfex/temperature/mescan-surfex-nordics-eca_csu.nc")
ds_id = xr.open_dataset("/data-uerra/mescan-surfex/temperature/mescan-surfex-nordics-eca_id.nc")
ds_su = xr.open_dataset("/data-uerra/mescan-surfex/temperature/mescan-surfex-nordics-eca_su.nc")

ds_cdd = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_cdd.nc")
ds_cwd = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_cwd.nc")
ds_pd = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_pd.nc")
ds_rr1 = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_rr1.nc")
ds_rx1day = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_rx1day.nc")
ds_rx5day = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_rx5day.nc")
ds_sdii = xr.open_dataset("/data-uerra/mescan-surfex/precipitation/mescan-surfex-nordics-eca_sdii.nc")

rasters = [ds_cfd, ds_fd, ds_csu, ds_id, ds_su, ds_cdd, ds_cwd, ds_pd, ds_rr1, ds_rx1day, ds_rx5day, ds_sdii]

Loop over the raster datasets and extract pixel values of the sample plots

In [15]:
rows = gdf.shape[0]
rows

61832

In [17]:
lons = [g.x for g in gdf['geometry'].tolist()]
lats = [g.y for g in gdf['geometry'].tolist()]

Loop over the sample plots and extract pixel values from the rasters

In [20]:
results = {}
for i, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    lon = row['geometry'].x
    lat = row['geometry'].y
    for ds in rasters:
        for v in ds.variables:
            if 'height' in ds.variables:
                heights = ds['height']
            times = ds['time']

            if v not in ["time", "lon", "lat", "inate.", "oordinate.", "height"]:
                if v not in results:
                    results[v] = []
                if 'height' in ds.variables:
                    values = ds.sel(time=times[0], height=heights[0], lon=lon, lat=lat, method='nearest')[v].values.tolist()
                else:
                    values = ds.sel(time=times[0], lon=lon, lat=lat, method='nearest')[v].values.tolist()
                results[v].append(values)

100%|██████████| 61832/61832 [1:14:29<00:00, 13.90it/s]


In [21]:
for k in results:
    gdf[f'uerra_{k}'] = results[k]

In [22]:
gdf.head()

,sampleplotid,clusterid,sampleplottype,measurementdate,center_x,center_y,center_z,mean_age_years,mean_dbh_cm,mean_height_m,...,uerra_consecutive_dry_days_index_per_time_period,uerra_number_of_cdd_periods_with_more_than_5days_per_time_period,uerra_consecutive_wet_days_index_per_time_period,uerra_number_of_cwd_periods_with_more_than_5days_per_time_period,uerra_precipitation_days_index_per_time_period,uerra_wet_days_index_per_time_period,uerra_highest_one_day_precipitation_amount_per_time_period,uerra_highest_five_day_precipitation_amount_per_time_period,uerra_number_of_5day_heavy_precipitation_periods_per_time_period,uerra_simple_daily_intensity_index_per_time_period
0,639971,48070005,1,2017-08-14,393888.00,7401168.00,0.00,39,9.20,9.10,...,29.0,209.0,15.0,36.0,1892.0,1892.0,33.170521,33.170521,0.0,5.050500
1,718519,47110091,1,2018-07-18,576046.89,7395714.46,346.79,66,16.41,9.95,...,24.0,166.0,14.0,43.0,2025.0,2025.0,49.622128,49.622128,0.0,4.469509
2,104636,40110069,1,2011-09-26,573195.59,7037602.47,197.03,72,23.75,17.73,...,20.0,196.0,13.0,56.0,2074.0,2074.0,53.007542,53.007542,1.0,5.247389
3,265558,43116119,1,2013-08-29,594209.26,7181267.18,255.95,42,17.99,14.19,...,25.0,174.0,12.0,58.0,2158.0,2158.0,43.480469,43.480469,0.0,5.019283
4,696784,39100744,1,2018-08-13,546924.17,6997684.98,124.56,14,6.95,7.19,...,47.0,214.0,13.0,32.0,1844.0,1844.0,60.879169,60.879169,1.0,5.115222


In [23]:
gdf.to_csv('../../data/finland/new_sample_plots_uerra_cdf_features.csv', index=False)

Load ERA5 CDO indexes

In [26]:
ds_cfd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacfd.nc")
ds_fd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecafd.nc")
ds_csu = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacsu.nc")
ds_id = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecaid.nc")
ds_su = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecasu.nc")

ds_cdd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacdd.nc")
ds_cwd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecacwd.nc")
ds_pd = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecapd.nc")
ds_rr1 = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarr1.nc")
ds_rx1day = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarx1day.nc")
ds_rx5day = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecarx5day.nc")
ds_sdii = xr.open_dataset("/data-uerra/era5/cdo/era5-nordics-ecasdii.nc")

rasters = [ds_cfd, ds_fd, ds_csu, ds_id, ds_su, ds_cdd, ds_cwd, ds_pd, ds_rr1, ds_rx1day, ds_rx5day, ds_sdii]

In [27]:
ds_cdd

<xarray.Dataset>
Dimensions:                                                     (latitude: 73, longitude: 111, time: 1)
Coordinates:
  * time                                                        (time) datetime64[ns] 2018-09-01T03:00:00
  * longitude                                                   (longitude) float32 4.5 ... 32.0
  * latitude                                                    (latitude) float32 71.5 ... 53.5
Data variables:
    consecutive_dry_days_index_per_time_period                  (time, latitude, longitude) float32 ...
    number_of_cdd_periods_with_more_than_5days_per_time_period  (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Wed Jan 09 14:42:55 2019: cdo eca_cdd era5-nordics-tp_daily...
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [28]:
results = {}
for i, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    lon = row['geometry'].x
    lat = row['geometry'].y
    for ds in rasters:
        for v in ds.variables:
            if 'height' in ds.variables:
                heights = ds['height']
            times = ds['time']

            if v not in ["time", "longitude", "latitude", "inate.", "oordinate.", "height"]:
                if v not in results:
                    results[v] = []
                if 'height' in ds.variables:
                    values = ds.sel(time=times[0], height=heights[0], longitude=lon, latitude=lat, method='nearest')[v].values.tolist()
                else:
                    values = ds.sel(time=times[0], longitude=lon, latitude=lat, method='nearest')[v].values.tolist()
                results[v].append(values)

100%|██████████| 61832/61832 [1:02:50<00:00, 16.40it/s]


In [31]:
for k in results:
    print(k)
    gdf[f'era5_{k}'] = results[k]

number_of_cfd_periods_with_more_than_5days_per_time_period
frost_days_index_per_time_period
number_of_csu_periods_with_more_than_5days_per_time_period
ice_days_index_per_time_period
summer_days_index_per_time_period
consecutive_dry_days_index_per_time_period
number_of_cdd_periods_with_more_than_5days_per_time_period
consecutive_wet_days_index_per_time_period
number_of_cwd_periods_with_more_than_5days_per_time_period
precipitation_days_index_per_time_period
wet_days_index_per_time_period
highest_one_day_precipitation_amount_per_time_period
highest_five_day_precipitation_amount_per_time_period
number_of_5day_heavy_precipitation_periods_per_time_period
simple_daily_intensity_index_per_time_period


In [33]:
gdf.to_csv('../../data/finland/new_sample_plots_all_features.csv', index=False)